Overall aim is to use the grant scheme stats in combination with public available charging points to assign EV charge points to local authorities.

In [1]:
import os
import json
os.chdir("../..")
from pipelines.util import *

In [2]:
def filter_local_authorities(df, colname):
    '''
    Extracts LAs only from a dataframe with arbitrary GSS codes.

    Parameters
    ---
    df: dataframe

    colname: the name containing GSS codes.
    '''
    assert colname in df.columns, f"No column called {colname} in dataframe"
    result = df[df[colname].str.match('^(E06|E07|E08|E09|S12|W06|N09)([0-9]{6})')]
    return result

def local_authority_to_constituency(data, floor=True):
    '''
    Map LAs to constituencies using a lookup of overlaps.
    '''
    with open('lookups/lookup-LAD24CD-PCON24CD.json') as f:
        lookup = json.load(f)

    # create an empty dataframe to put the data into
    df = pd.DataFrame(columns=['PCON24CD', 'workplace-jan-25', 'home-2022'])
    df.set_index('PCON24CD', inplace=True)

    # iterate through each column of the LA_Code dataframe
    for la, c in lookup.items():
        # get a key and value pair for each constituency and its overlap
        for constituency_code, overlap in c.items():
            # if constituency doesn't exist, loop over columns and set all to 0.
            if constituency_code not in df.index.to_list():
                for cname in df.columns.to_list():
                    df.loc[constituency_code, cname] = 0.0
            for col in df.columns.to_list():
                # try:
                #     # calculate the share of charging devices in each constituency
                share = data.loc[data['ONS LA Code']==la, col].values[0]*overlap
                df.loc[constituency_code, col] += share
                # except:
                #     print('failed')
        
    if floor==True:
        df = df.apply(np.floor)

    return df

def fix_ons_codes(old_codes, new_code, df, code_col, value_col):
    '''
    Params
    ---

    old_codes: list of the old codes to fix.

    new_code: string for code that replaces old codes.

    df: dataframe.

    code_col: name of the column in the df that has the GSS codes.

    value_col: name of the column containing the values to add up.
    '''
    # Replace the list with the ONS codes you want to sum
    sum_value = df[df[code_col].isin(old_codes)][value_col].sum()

    # Create a new row with the new ONS code and the calculated sum
    new_row = pd.DataFrame({f"{code_col}": [new_code], f"{value_col}": [sum_value]})

    # Append the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    return df

Get the total number of workplace sockets installed by local authority. These use the latest LA codes (2023/24 - 361).

In [3]:
workplace_charging = read_data("raw-data/electric-vehicle-charging-device-grant-scheme-statistics-january-2025.ods", sheet_name='4', engine='odf', skiprows=2)
workplace_charging = filter_local_authorities(workplace_charging, 'ONS LA Code')
workplace_charging = workplace_charging.loc[:, ['ONS LA Code', 'Sockets Installed']]
workplace_charging.rename(columns={'Sockets Installed': 'workplace-jan-25'}, inplace=True)

Get the total number of home charging "devices" (could these have >1 sockets?) by local authority. These use old LA codes as so some fixes are applied.
See <https://en.wikipedia.org/wiki/2019%E2%80%932023_structural_changes_to_local_government_in_England>

In [4]:
home_charging = read_data("raw-data/electric-vehicle-charging-device-grant-scheme-statistics-january-2025.ods", sheet_name='13', engine='odf', skiprows=2)
home_charging = filter_local_authorities(home_charging, 'ONS LA Code')
home_charging = home_charging.loc[:, ['ONS LA Code', 'Grand Total']]
home_charging.rename(columns={'Grand Total': 'home-2022'}, inplace=True)
# Allerdale, Carlisle and Copeland are now cumbria.
home_charging = fix_ons_codes(old_codes=['E07000026', 'E07000028', 'E07000029'], 
                              new_code='E06000063', 
                              code_col='ONS LA Code', 
                              df=home_charging, 
                              value_col='home-2022')
# Barrow in furness, eden and south lakeland are now westmorland and furness
home_charging = fix_ons_codes(old_codes=['E07000027', 'E07000030', 'E07000031'], 
                              new_code='E06000064', 
                              code_col='ONS LA Code', 
                              df=home_charging, 
                              value_col='home-2022')
# North yorkshire replaced the 7 north yorkshire LAs.
home_charging = fix_ons_codes(old_codes=['E07000163','E07000164','E07000165','E07000166','E07000167','E07000168','E07000169'], 
                              new_code='E06000065', 
                              code_col='ONS LA Code', 
                              df=home_charging, 
                              value_col='home-2022')
# Somerset replaced Mendip, Sedgemoor, Somerset West and Taunton, and South Somerset
home_charging = fix_ons_codes(old_codes=['E07000187','E07000188','E07000246','E07000189',], 
                              new_code='E06000066', 
                              code_col='ONS LA Code', 
                              df=home_charging, 
                              value_col='home-2022')

### Map 2024 LADs PCON24CD

In [5]:
home_and_workplace = pd.merge(home_charging, workplace_charging)
# merged['total'] = merged[['workplace-jan-25', 'home-2022']].sum(axis=1)
home_and_workplace = local_authority_to_constituency(home_and_workplace)
home_and_workplace

,workplace-jan-25,home-2022
PCON24CD,,
E14001272,53,375
E14001367,76,253
E14001368,116,511
E14001440,25,140
E14001518,138,482
...,...,...
W07000088,41,284
W07000105,59,414
W07000109,46,303


### Public charging
This has kindly been processed. Combining this with the current data only requires a small amount of renaming.

In [6]:
public = pd.read_csv("src/themes/transport/ev-charging-points/_data/release/electric-vehicle-public-charging-infrastructure-statistics-january-2025-table-7a.csv",
                     usecols=['Parliamentary constituency Code', 'Jan-25'])
public.rename(columns={'Jan-25': 'public-jan-25', "Parliamentary constituency Code": 'PCON24CD'}, inplace=True)
public.set_index("PCON24CD", inplace=True)

Now merge the public, home, and workplace data

In [7]:
result = public.merge(home_and_workplace, on='PCON24CD')
result['total'] = result.sum(axis=1)
result.to_csv("src/themes/transport/ev-charging-points/_data/release/ev_charging_points.csv")